In [6]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import csv
import os



def scrape_page2(url, dict_data):
    
    table = main_html_parser(url)
    
    hcpcs_data = []
    
    # Find the first row in the table
    first_row = table.find('tr')
    
    if first_row:
        cells = first_row.find_all('td')
        
    if len(cells) == 2:  
        code = cells[0].get_text(strip=True)
        desc = cells[1].get_text(strip=True)
        dd = {'Short Description': desc}
        dd.update(dict_data)
        hcpcs_data.append(dd)
    
    
    
    # Takes Headers for csv columns
    if hcpcs_data:  # Ensure hcpcs_data is not empty
        headers = list(hcpcs_data[0].keys())
    else:
        headers = []
        
    filename = "scrapped_data.csv"
    
    # Saving scrapped data into csv file
    
    # Check if the file exists
    file_exists = os.path.isfile(filename)
    
    # Write the list of dictionaries to the CSV file
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        
        # If the file doesn't exist, write the headers
        if not file_exists:
            writer.writeheader()

        # Write rows
        for row in hcpcs_data:
            writer.writerow(row)
            
    return hcpcs_data

            

def scrape_page1(url, dict_data):
    
    # Calling html parsing function
    table = main_html_parser(url)
    
    hcpcs_data = []
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) == 2:  
            code = cells[0].get_text(strip=True)
            desc = cells[1].get_text(strip=True)
            link = urllib.parse.urljoin(url, row.find('a')['href']) 
            dd = {'Code': code, 'Long Description': desc}
            dd.update(dict_data)
            page_data = scrape_page2(link,dd)
#             hcpcs_data = hcpcs_data + page_data

    return page_data



def scrape_page(url):
    
    # Calling html parsing function
    table = main_html_parser(url)
    
    # Extract data from the table
    hcpcs_data = []
    
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) == 3:  # Ensure it's a valid row with all necessary data
            code = cells[0].get_text(strip=True)
            description = cells[2].get_text(strip=True)
            link = urllib.parse.urljoin(url, row.find('a')['href'])  
            dictdata = {'Group': code,'Category': description}
            page_data = scrape_page1(link,dictdata)
#             hcpcs_data = hcpcs_data + page_data

   
    return "Completed..."


# Function for getting url response and parsing html page
def main_html_parser(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table containing the procedure codes
    table = soup.find('table', class_='table')
    
    return table
    
    
    
# Main scraping function
def main():
    base_url = 'https://www.hcpcsdata.com/Codes'
    
    page_data = scrape_page(base_url)
    
    print(page_data)

if __name__ == "__main__":
    main()


KeyboardInterrupt: 